### Import libraries and data

In [1]:
import pandas as pd
import csv
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
vols = pd.read_csv("volumes300.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [19]:
scaledvols = pd.read_csv("ScaledVolumes.csv")
scaledvols = scaledvols.drop("Unnamed: 0", axis=1)

In [6]:
temps = pd.read_csv("temperatures300.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [7]:
#headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bi", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
#"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
#"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
#"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu", "La", "Li", "Tm", "Ti", "Te", "Tb", "Tc", "Yb", "Dy",\
#"I", "Y", "Ag", "Al", "As", "In"]

headers = ["H","Li","Be","B","C","N","O","F","Na","Mg","Al","Si","P","S","Cl",\
"K","Ca","Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Ga",\
"Ge","As","Se","Br","Rb","Sr","Y","Zr","Nb","Mo",\
"Tc","Ru","Rh","Pd","Ag","Cd","In","Sn",\
"Sb","Te","I","Xe","Cs","Ba","La","Ce","Pr",\
"Nd","Sm","Eu","Gd","Tb","Dy","Ho","Er",\
"Tm","Yb","Lu","Hf","Ta","W","Re","Os","Ir",\
"Pt","Au","Hg","Tl","Pb","Bi","Ac","Th","Pa",\
"U","Np","Am"] #hofmann

#headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bi", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
#"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
#"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
#"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu"]


formula = pd.read_csv("formulae.csv", usecols=headers)
#unitformula = unitformula.drop("Unnamed: 0", axis=1)

It is important to note that this was just an attempt to perform the calculations in a different manner; where the volumes are scaled to the unit cell volume rather than the formulae.

### Setting models

In [15]:
from sklearn import linear_model, svm, kernel_ridge, ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict

### Calculations & Graphs

In [ ]:
estim = [(linear_model.RANSACRegressor(linear_model.LinearRegression())),\
          (linear_model.RANSACRegressor(linear_model.Ridge(alpha=100))), (RandomForestRegressor(n_estimators=100, n_jobs=-1)),\
          (ensemble.GradientBoostingRegressor(n_estimators=100)), (ensemble.ExtraTreesRegressor(n_estimators=100, n_jobs=-1))]

for reg in estim:
    print "\nUsing: ", reg
    scores = cross_val_score(reg, formula, scaledvols)
    pred = cross_val_predict(reg, formula, scaledvols)
    print "\nMean: ", scores.mean(), "STD: ", scores.std()
    predDF = pd.DataFrame(pred, columns = ["Pred Vol"])
    predDF["Pred Vol"].div(scaledvols["ScaledVolumes"]).hist(bins=50)
    print predDF["Pred Vol"].div(scaledvols["ScaledVolumes"]).describe()
    plt.show() 

As always the models are scored based on their ability to predict - this time the scaled volumes. Also graphs are made to show the ratio between the predicted volumes and actual ones.

In [ ]:
estim = [(linear_model.RANSACRegressor(linear_model.LinearRegression())),\
          (linear_model.RANSACRegressor(linear_model.Ridge(alpha=100))), (RandomForestRegressor(n_estimators=100, n_jobs=-1)),\
          (ensemble.GradientBoostingRegressor(n_estimators=100)), (ensemble.ExtraTreesRegressor(n_estimators=100, n_jobs=-1))]
X_train, X_test, y_train, y_test = train_test_split(formula, scaledvols, random_state=24)
for reg3 in estim:
    #X_train, X_test, y_train, y_test = train_test_split(unitformula, vols, random_state = 24)
    reg3.fit(X_train, y_train)
    pdv = reg3.predict(X_test)
    print "Using: ", reg3
    print reg3.score(X_test,y_test)
    pdvd = pd.DataFrame(pdv, columns = ["Pred Vol"])
    ytest = pd.DataFrame(y_test)
    ytest = ytest.reset_index(drop=True)
    pdvd["Pred Vol"].div(ytest["ScaledVolumes"]).hist(bins=50)
    print pdvd["Pred Vol"].div(ytest["ScaledVolumes"]).describe()
    plt.show()

This is the same idea but TTS is used; just to check if the scores do change - they do but not by much.
This did cause shape errors initially

In [ ]:
estim = [(linear_model.RANSACRegressor(linear_model.LinearRegression())),\
          (linear_model.RANSACRegressor(linear_model.Ridge(alpha=100))), (RandomForestRegressor(n_estimators=100, n_jobs=-1)),\
          (ensemble.GradientBoostingRegressor(n_estimators=100)), (ensemble.ExtraTreesRegressor(n_estimators=100, n_jobs=-1))]

for reg2 in estim:
    reg2.fit(formula, scaledvols)
    print "Using ",reg2
    print reg2.score(formula,scaledvols)
    pred2 = reg2.predict(formula)
    pred2DF = pd.DataFrame(pred2, columns = ["Pred Vol"])
    pred2DF["Pred Vol"].div(scaledvols["ScaledVolumes"]).hist(bins=50)
    print pred2DF["Pred Vol"].div(scaledvols["ScaledVolumes"]).describe()
    plt.show()

This is an attempt without cross validation or TTSp; again just to see if the scores change.